<h2>Question #1</h2>

In [14]:
#Importing librares
import pandas as pd
import datetime
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
import plotly_express as px
from sklearn.metrics import silhouette_score, accuracy_score
from sklearn.preprocessing import StandardScaler
import shap
import matplotlib as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots


#importing the CSV
data=pd.read_csv('./marketing_campaign.csv',delimiter='\t')

In [15]:
#Dropping the ID column, rows with empty cells & duplicates
data.drop('ID',axis=1,inplace=True)
data.dropna(inplace=True)
data.drop_duplicates(inplace=True)
data.reset_index(drop=True,inplace=True)
cols = data.columns

In [16]:
#Converting Education & Marital_Status columns from object(string) fields into integer-based categorical columns
data['Education']=pd.Categorical(data['Education'])
data['Education']=data['Education'].cat.codes

data['Marital_Status']=pd.Categorical(data['Marital_Status'])
data['Marital_Status']=data['Marital_Status'].cat.codes

In [17]:
#Converting the Dt_Customer column from object(string) fields into integer-based number of days being a customer
today= datetime.datetime.now()

for i in range(len(data)):
    temp_dt = today - pd.to_datetime(data['Dt_Customer'][i])
    data['Dt_Customer'][i]=temp_dt.days

data['Dt_Customer'] = data['Dt_Customer'].astype(int)
data['Year_Birth'] = data['Year_Birth'].astype(int)

for i in range(len(data)):
    data['Year_Birth'][i]=2021-data['Year_Birth'][i]


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
data

,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,MntFruits,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,64,2,4,58138.0,0,0,3496,58,635,88,...,7,0,0,0,0,0,0,3,11,1
1,67,2,4,46344.0,1,1,2650,38,11,1,...,5,0,0,0,0,0,0,3,11,0
2,56,2,5,71613.0,0,0,2997,26,426,49,...,4,0,0,0,0,0,0,3,11,0
3,37,2,5,26646.0,1,0,2590,26,11,4,...,6,0,0,0,0,0,0,3,11,0
4,40,4,3,58293.0,1,0,2846,94,173,43,...,5,0,0,0,0,0,0,3,11,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2029,44,2,5,666666.0,1,0,3193,23,9,14,...,6,0,0,0,0,0,0,3,11,0
2030,54,2,3,61223.0,0,1,3066,46,709,43,...,5,0,0,0,0,0,0,3,11,0
2031,40,2,2,56981.0,0,0,2840,91,908,48,...,6,0,1,0,0,0,0,3,11,0
2032,65,3,5,69245.0,0,1,2841,8,428,30,...,3,0,0,0,0,0,0,3,11,0


In [19]:
#Shuffling the dataframe, just in case ;)
data=data.sample(frac=1).reset_index(drop=True)

In [20]:
#Calculating Silhouette Score for range of values of K to find out the most optimal K
scores=[]
for k in range(2,10):
    model=KMeans(n_clusters=k,random_state=0).fit(data)
    scores.append(silhouette_score(data,model.labels_))

In [21]:
#Plotting a graph for Silhouette Score of each K
px.line(y=scores,x=range(2,10),title='Silhouette Score for each K',labels={
    'x':'Values of K',
    'y':'silhouette_score'
})

<h2>Thus, lookin at the above graphs it can be concluded that the most optimal score of K is 3, since the silouhette score is the maximum when k=3</h2>

In [31]:
K=3
model=KMeans(n_clusters=K,random_state=0).fit(data)
data['Cluster']=model.labels_
centers=model.cluster_centers_

In [226]:
fig=px.pie(values=data['Cluster'].value_counts(),names=['Cluster 1', 'Cluster 2','Cluster3'],title='Number of people per Cluster')
fig.update_traces(
    textinfo='percent+value+label'
)
fig.show()

<h2>From the above, graph, it can be inferred that both groups consist of significant amount of people and therefore the brand would have to focus on both equally.</h2>

In [211]:
tempdf=pd.DataFrame({
    'Spending':data['MntFruits']+data['MntGoldProds']+data['MntFishProducts']+data['MntMeatProducts']+data['MntWines']+data['MntSweetProducts'],
    'Cluster':data['Cluster']
})

px.scatter(tempdf,y='Spending',color='Cluster',trendline='lowess',title='Total Spending in each cluster',marginal_y='box')

<h2>From the above graph, it can be inferred that people in Cluster2 are spending more than most people and have spent above 550$ in the last 2 years.</h2>
<p>Let us find more about them.</p>

In [136]:
tempdf=pd.DataFrame({
    'Income':data['Income'],
    'Age':data['Year_Birth'],
    'Cluster':data['Cluster']
})

px.scatter(tempdf,x='Age',y='Income',color='Cluster',
range_x=(20,90),range_y=(0,150000),trendline='expanding',
title='Income v/s Age per Cluster', marginal_x='box')

<h2>From the above graph, it can be inferred that people in Cluster2 are have higher income and earn mostly above 47K.<br>
Also, people of all age groups belong to both the clusters.</h2>

In [208]:
tempdf=pd.DataFrame({
    'Fruits':data['MntFruits'],
    'Gold':data['MntGoldProds'],
    'Meat':data['MntMeatProducts'],
    'Wine':data['MntWines'],
    'Sweets':data['MntSweetProducts'],
    'Spending':data['MntFruits']+data['MntGoldProds']+data['MntFishProducts']+data['MntMeatProducts']+data['MntWines']+data['MntSweetProducts'],
    'Cluster':data['Cluster']
})


specs = [[{'type':'domain'}, {'type':'domain'}], 
         [{'type':'domain'}, {'type':'domain'}],
         [{'type':'domain'}, {'type':'domain'}]
        ]

fig = make_subplots(rows=3, cols=2, specs=specs,subplot_titles=['Fruits', 'Gold','Meat','Wine','Sweets'])


fig.add_trace(go.Pie(values=tempdf['Fruits'],labels=tempdf['Cluster']),1,1)
fig.add_trace(go.Pie(values=tempdf['Gold'],labels=tempdf['Cluster']),1,2)
fig.add_trace(go.Pie(values=tempdf['Meat'],labels=tempdf['Cluster']),2,1)
fig.add_trace(go.Pie(values=tempdf['Wine'],labels=tempdf['Cluster']),2,2)
fig.add_trace(go.Pie(values=tempdf['Sweets'],labels=tempdf['Cluster']),3,1)
fig.add_trace(go.Pie(values=tempdf['Sweets'],labels=tempdf['Cluster']),3,2)

fig = go.Figure(fig)
fig.show()


<h2>In Cluster2, least number of people spend on Gold, while most spend on Meat.<br>
In Cluster1, least number of people spend on Meat, while most spend on Gold.</h2>
<p>It is surpising to see that both trends are completely opposite.<br>Cluster2 also does 86% of the spending.</p>

<h2>FINAL ANSWER:<br></h2>
<h4>Thus, the market can be divided into two groups.<br>
<b>Group 1</b> is the less spending one, the people have incomes lesser than 50K and most have spent less than 400 in the last 2 years. Quiet a number of people in group 1 spend on Gold.<br>

<b>Group 2</b> is the more spending one, the people have higher incomes and therefore, spend more. They do around 86% of the total spending and significantly purchase most of the products offered. They did 86% of the spending within the last 2 years.
</h4>

<h2>Question #2</h2>
<p>For this part of the problem, I will be performing multinomial classification on the promotions section of the data.<br>
I will use a XGBoost Classifier to classify if a customer is one of the following:<br><br>

1. Will part in no promotions.<br>
2. Will part in one promotion.<br>
3. Will part in more than one promotions.<br><br>

Since I am doing multimodal classification with 3 classes, I will have 3 different SHAP analyses.
</p>

In [24]:
#Dropping the Cluster Column for this part of the problem
data.drop('Cluster',axis=1,inplace=True)

In [25]:
#Extracting data required specifically for this problem statement from DATA df

#List of promo columns.
promos=['Response','AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4',
       'AcceptedCmp5']

#List of columns excluding promo columns
xgcols=[]
for col in cols:
    if col not in promos:
        xgcols.append(col)

#List of TRUE label i.e. the number of times one interacted with a promotion
xglabels=[]

for i in range(len(data)):
    temp = 0
    for j in range(len(promos)):
        if data[promos[j]][i]==1:
            temp+=1

    if temp==0:
        xglabels.append(0)
    elif temp==1:
        xglabels.append(1)
    else:
        xglabels.append(2)

In [26]:
#Initializing and training a XGBClassifier
from xgboost import XGBClassifier

model = XGBClassifier(
    base_score=0.6,
    use_label_encoder=False,
    random_state=0,
    n_estimators=100,
    n_jobs=10,
    learning_rate=0.05,
    verbosity=0,
    max_depth=10,
    booster='gbtree',
    tree_method = 'exact',
    objective='multi:softprob',
    num_class=3,
    sub_sample=0.33,
    random_seed=0
    )

ntrain = 7*len(data)//10    #The number of data samples to use for training

model.fit(data[xgcols][:ntrain],xglabels[:ntrain])

preds=model.predict(data[xgcols][ntrain:])
print(accuracy_score(preds,xglabels[ntrain:]))  #Final Validation accuracy of the model

0.7364975450081833


In [27]:
#Performing SHAP Analysis
shap.initjs()
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(data[xgcols])   #Generating the SHAP values

C:\Users\naman\.conda\envs\mi\lib\site-packages\xgboost\core.py:105: UserWarning:

ntree_limit is deprecated, use `iteration_range` or model slicing instead.



In [28]:
#SHAP analysis for people who did not interact with any promotions
shap.force_plot(base_value=explainer.expected_value[0], shap_values=shap_values[0][0,:], features=xgcols,show=True)

<h2>For people who did not use any coupons<br></h2>
<h4>In the above SHAP analysis, the factor most significantly influencing the outcome is the Number of Website visits per month. And, it is surprising to see that it is influnceing people to not take part in promotions. Perhaps, the website is poor or does not provide adequate details about the promotions going on. Therfore, the website must be looked at.<br><br>
It can also be inferred that the promotions are failing to attract people who aren't our recent customers. Since it can be inferred that, the higher the recency(older customer) of the customer, the less likely he is going to interact with the promotions. Perhaps the promotions are failing to attract old customers. It is also followed by Dt_customer, adding to the argument.<br>
</h4>

In [29]:
#SHAP analysis for people who did not interact with one promotion
shap.force_plot(base_value=explainer.expected_value[1], shap_values=shap_values[1][0,:], features=xgcols,show=True)

<h2>For people who used only one coupon<br></h2>
<h4>From the SHAP analysis, it can be firstly inferred that, people who spend on wine are most definately going to apply a coupon. It could be the case that the brand's wine promotions are doing really well and the brand could apply similar strategies to other products as well.<br>
Second observation, strenghtens our inference from the previous analysis that the website is keeping customers from interacting with the customers. It seems like people who are not visitng the websites are more likely to interact with a promotion.<br><br>
It can also be noticed that, age, recency, income, sweets purchase, fruits purchase are also contributing negatively towards interaction of customer with a promotion.
</h4>

In [30]:
#SHAP analysis for people who did not interact with more than one promotion
shap.force_plot(base_value=explainer.expected_value[2], shap_values=shap_values[2][0,:], features=xgcols,show=True)

<h2>For people who use more than one coupon<br></h2>
<h4>From the SHAP analysis, it can be firstly inferred that, people who have greater income intercat more with the promotions! Perhaps it could be the secret to their wealth or maybe that our promotions are only more suitable to those who have higher income.<br><br>
It can also be seen that people who spend money on gold, refrain from interacting with more than one promotion. 

It can be noted that people who more recently made a purchase with the brand are more likely to interact with more than one promotion.

</h4>

<h1>Final Answer</h1>
<h4>
1. The website is driving people away from the promtions. It should be looked into immediately.<br><br>
2. The promotions are significantlyly failing to attract people who once made a purchase with us. Therefore, they are failing to attract our old customers.<br><br>
3. It must be noted that people purhasing wine are interacting with the promotions the most. A root-cause analysis should be done to better understand why people purchasing wine are more attracted to our promotions.<br><br>
4. It seems that only people with good income are interacting witht he promotions, it could be that our promotions are not suitable to people with lesser income. Therefore, research is warranted here.
5. It also seems that our promotions are successfull in capturing customers who have recently made purchases with us and they are responsing to the campaigns multiple times.
</h4>